In [2]:
options(dplyr.summarise.inform = FALSE)
library(tidyverse)
library(data.table)

In [3]:
source(paste0(dirname(getwd()),'/map.r'))

#### 0 - Read collected LILAC data

In [4]:
lilac_qc <- fread( paste0(TMP_DIR, "lilac_qc.csv"))
lilac <- fread( paste0(TMP_DIR, "lilac.csv")) %>% filter(sampleId %in% (lilac_qc %>% filter(Status == "PASS") %>% pull(sampleId)))

#### 1 - Prep biomarkers
- What to include, derive, filter?
- HLA heterozygosity?
- HLA LOH?

In [5]:
hlas <- 
lilac |> 
  rowwise() |> 
  mutate( type = strsplit(Allele, ":")[[1]][1]) |>
  group_by( sampleId, type ) %>% 
  summarise( cn = sum(TumorCopyNumber)) |>
  spread(type, cn)

In [6]:
hlas[is.na(hlas)] <- 0

In [7]:
colnames(hlas) <- c("sampleId", paste0("lilac_", colnames(hlas)[-1]))

In [8]:
hlas

sampleId,lilac_A*01,lilac_A*02,lilac_A*03,lilac_A*11,lilac_A*23,lilac_A*24,lilac_A*25,lilac_A*26,lilac_A*29,⋯,lilac_C*05,lilac_C*06,lilac_C*07,lilac_C*08,lilac_C*12,lilac_C*14,lilac_C*15,lilac_C*16,lilac_C*17,lilac_C*18
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ACTN01020001T,0.00,0.98,1.97,0.00,0,0.00,0,0.00,0.00,⋯,0.00,0.00,1.97,0.00,0.00,0.00,0.00,0,0.00,0
ACTN01020002T,0.00,0.00,1.02,0.00,0,0.00,0,0.00,0.00,⋯,0.00,0.00,0.00,0.00,1.01,0.00,0.00,0,0.00,0
ACTN01020003T,0.00,0.00,0.90,0.00,0,0.00,0,0.00,0.00,⋯,0.00,1.02,0.00,0.00,0.00,0.00,0.00,0,0.90,0
ACTN01020005T,0.00,1.00,0.00,0.00,0,1.02,0,0.00,0.00,⋯,0.00,2.02,0.00,0.00,0.00,0.00,0.00,0,0.00,0
ACTN01020006T,0.00,0.00,3.57,0.00,0,0.00,0,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,2.58,0.00,0,0.00,0
ACTN01020007T,0.00,1.09,0.99,0.00,0,0.00,0,0.00,0.00,⋯,0.00,0.00,2.08,0.00,0.00,0.00,0.00,0,0.00,0
ACTN01020009T,0.00,2.38,0.00,0.00,0,0.00,0,0.00,0.00,⋯,1.43,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0
ACTN01020010T,1.96,0.00,0.00,0.00,0,0.00,0,3.96,0.00,⋯,0.00,0.00,5.92,0.00,0.00,0.00,0.00,0,0.00,0
ACTN01020011T,0.00,1.02,0.02,0.00,0,0.00,0,0.00,0.00,⋯,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0


#### 2 - Send it!

In [9]:
fwrite(hlas, paste0(READY_DIR, "lilac_ready.csv"))